In [46]:
from neo4j import GraphDatabase
from py2neo import Graph
import pandas as pd
import numpy as np

In [47]:
uri="neo4j+s://1c8f07f5.databases.neo4j.io"
username="neo4j"
password="uycJUUxlA1SDUncKIgImCp_DT8hASOiqSQ-V7pnLRCY"
graph = GraphDatabase.driver(uri, auth=(username, password))

In [48]:
ratings_test = pd.read_csv('data\\ratings_test.csv')
user_ids = ratings_test['userId'].unique()

In [49]:
def make_recommendations_collaborative_filtering(id, uri, username, password):
    graph = GraphDatabase.driver(uri, auth=(username, password))
    # User ID for whom you want to get recommendations
    target_user_id = id

    # Create a session
    with graph.session() as session:
        # Execute the Cypher query for recommendations
        query = """
        MATCH (targetUser:User {id: $target_user_id})-[:watched]->(ratedMovie:Movie)<-[:watched]-(similarUser:User)-[:watched]->(recommendedMovie:Movie)
        WHERE NOT (targetUser)-[:watched]->(recommendedMovie)
        WITH recommendedMovie, AVG(similarUser.popularity) AS predictedPopularity
        RETURN recommendedMovie, predictedPopularity
        ORDER BY predictedPopularity DESC
        LIMIT 20
        """
        result = session.run(query, target_user_id=target_user_id)

        liste = []
        # Process the result and print or use the recommendations
        for record in result:
            recommended_movie = record["recommendedMovie"]
            liste.append(recommended_movie["original_title"])

    # Close the Neo4j connection (not necessary with 'with' statement)
    graph.close()
    
    return liste

In [50]:
def make_recommendations_collaborative_popularitybased(id, uri, username, password):
    graph = GraphDatabase.driver(uri, auth=(username, password))
    # User ID for whom you want to get recommendations
    target_user_id = id

    # Create a session
    with graph.session() as session:
        # Execute the Cypher query for recommendations
        query = """
        MATCH (targetUser:User {id: $target_user_id})-[:watched]->(ratedMovie:Movie)<-[:watched]-(similarUser:User)-[:watched]->(recommendedMovie:Movie)
        WHERE NOT (targetUser)-[:watched]->(recommendedMovie)
        WITH recommendedMovie
        order by recommendedMovie.popularity desc
        RETURN distinct recommendedMovie
        LIMIT 20
        """
        result = session.run(query, target_user_id=target_user_id)

        liste = []
        # Process the result and print or use the recommendations
        for record in result:
            recommended_movie = record["recommendedMovie"]
            liste.append(recommended_movie["original_title"])

    # Close the Neo4j connection (not necessary with 'with' statement)
    graph.close()
    
    return liste

In [51]:
def make_recommendations_genre(id, uri, username, password):
    graph = GraphDatabase.driver(uri, auth=(username, password))
    # User ID for whom you want to get recommendations
    target_user_id = id

    # Create a session
    with graph.session() as session:
        # Execute the Cypher query for recommendations
        query = """
        MATCH (targetUser:User {id: $target_user_id})-[:watched]->(:Movie)-[:IN_GENRE]->(favoriteGenre:Genre)
        WITH targetUser, favoriteGenre, COUNT(*) AS genreCount
        ORDER BY genreCount DESC
        WITH targetUser, COLLECT(favoriteGenre)[0] AS favoriteGenre
        MATCH (recommendedMovie:Movie)-[:IN_GENRE]->(favoriteGenre)
        WHERE NOT (targetUser)-[:watched]->(recommendedMovie)
        WITH recommendedMovie
        ORDER BY recommendedMovie.popularity DESC
        RETURN recommendedMovie, recommendedMovie.popularity
        LIMIT 20;
        """
        result = session.run(query, target_user_id=target_user_id)

        liste = []
        # Process the result and print or use the recommendations
        for record in result:
            recommended_movie = record["recommendedMovie"]
            liste.append(recommended_movie["original_title"])

    # Close the Neo4j connection (not necessary with 'with' statement)
    graph.close()
    
    return liste

In [52]:
def make_recommendations_mostsimilar_knn(id, uri, username, password):
    graph = GraphDatabase.driver(uri, auth=(username, password))
    # User ID for whom you want to get recommendations
    target_user_id = id

    # Create a session
    with graph.session() as session:
        # Execute the Cypher query for recommendations
        query = """
        MATCH (targetUser:User {id: $target_user_id})-[:watched]->(commonMovie:Movie)<-[:watched]-(otherUser:User)-[:watched]->(m:Movie)
        WHERE NOT (m)<-[:watched]-(targetUser)
        WITH otherUser, COLLECT(DISTINCT m) AS commonMovies
        ORDER BY SIZE(commonMovies) DESC
        LIMIT 100

        UNWIND commonMovies AS commonMovie
        MATCH (otherUser:User)-[:watched]->(commonMovie)
        WITH commonMovie, COUNT(otherUser) AS userCount
        ORDER BY userCount DESC
        LIMIT 20

        RETURN commonMovie;
        """
        result = session.run(query, target_user_id=target_user_id)

        liste = []
        # Process the result and print or use the recommendations
        for record in result:
            recommended_movie = record["commonMovie"]
            liste.append(recommended_movie["original_title"])

    # Close the Neo4j connection (not necessary with 'with' statement)
    graph.close()
    
    return liste

In [59]:
def make_recommendations_favoritedirector(id, uri, username, password):
    graph = GraphDatabase.driver(uri, auth=(username, password))
    # User ID for whom you want to get recommendations
    target_user_id = id

    # Create a session
    with graph.session() as session:
        # Execute the Cypher query for recommendations
        query = """
        MATCH (targetUser:User {id: $target_user_id})-[:watched]->(commonMovie:Movie)<-[:DIRECTED]-(directors:Director)
        WITH targetUser, directors, COUNT(directors) AS dirCount
        ORDER BY dirCount DESC
        LIMIT 5

        unwind dirCount as dircount
        match (m)<-[:DIRECTED]-(directors)
        return m
        limit 20
        """
        result = session.run(query, target_user_id=target_user_id)

        liste = []
        # Process the result and print or use the recommendations
        for record in result:
            recommended_movie = record["m"]
            liste.append(recommended_movie["original_title"])

    # Close the Neo4j connection (not necessary with 'with' statement)
    graph.close()
    
    return liste

In [61]:
def make_recommendations_favoriteActor(id, uri, username, password):
    graph = GraphDatabase.driver(uri, auth=(username, password))
    # User ID for whom you want to get recommendations
    target_user_id = id

    # Create a session
    with graph.session() as session:
        # Execute the Cypher query for recommendations
        query = """
        MATCH (targetUser:User {id: $target_user_id})-[:watched]->(commonMovie:Movie)<-[:ACTED_IN]-(directors:Actor)
        WITH targetUser, directors, COUNT(directors) AS dirCount
        ORDER BY dirCount DESC
        LIMIT 5

        unwind dirCount as dircount
        match (m)<-[:ACTED_IN]-(directors)
        return m
        limit 20
        """
        result = session.run(query, target_user_id=target_user_id)

        liste = []
        # Process the result and print or use the recommendations
        for record in result:
            recommended_movie = record["m"]
            liste.append(recommended_movie["original_title"])

    # Close the Neo4j connection (not necessary with 'with' statement)
    graph.close()
    
    return liste

In [54]:
def precision_recall_f1_at_k(recommended_list, actual_list, k):
    # Take only the top-k recommended items
    recommended_list = recommended_list[:k]
    
    # Calculate the number of overlapping items between recommended and actual lists
    intersection = set(recommended_list) & set(actual_list)
    
    # Calculate precision
    precision = len(intersection) / len(recommended_list) if len(recommended_list) > 0 else 0.0
    
    # Calculate recall
    recall = len(intersection) / len(actual_list) if len(actual_list) > 0 else 0.0
    
    # Calculate F1 score
    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
    
    return [precision, recall, f1]

In [55]:
selected_users = [19, 182, 219, 274, 294, 480, 489, 597, 599, 600]

liste = []
for i in selected_users:
    recommended = make_recommendations_collaborative_filtering(id=i, uri=uri, username=username, password=password)
    liste.append(recommended)

array = []
for index, i in enumerate(selected_users):
    real = ratings_test[ratings_test['userId'] == i]['movieTitle'].tolist()
    reccomendation = liste[index]
    array.append(precision_recall_f1_at_k(reccomendation, real, 20))
    print(precision_recall_f1_at_k(reccomendation, real, 20))

array = np.array(array)
print("average precision is " + str(array[:,0].mean()) +", average recall is "+ str(array[:,1].mean())+ ", average f1 is " + str(array[:,2].mean()))

[0.3, 0.061855670103092786, 0.10256410256410256]
[0.15, 0.023076923076923078, 0.04]
[0.25, 0.06666666666666667, 0.10526315789473685]
[0.2, 0.020618556701030927, 0.037383177570093455]
[0.35, 0.11475409836065574, 0.1728395061728395]
[0.15, 0.025, 0.04285714285714286]
[0.25, 0.056179775280898875, 0.09174311926605504]
[0.3, 0.09836065573770492, 0.14814814814814814]
[0.35, 0.023178807947019868, 0.043478260869565216]
[0.2, 0.038834951456310676, 0.06504065040650406]
average precision is 0.25, average recall is 0.05285261053303035, average f1 is 0.08493172657491878


In [56]:
selected_users = [19, 182, 219, 274, 294, 480, 489, 597, 599, 600]

liste = []
for i in selected_users:
    recommended = make_recommendations_collaborative_popularitybased(id=i, uri=uri, username=username, password=password)
    liste.append(recommended)

array = []
for index, i in enumerate(selected_users):
    real = ratings_test[ratings_test['userId'] == i]['movieTitle'].tolist()
    reccomendation = liste[index]
    array.append(precision_recall_f1_at_k(reccomendation, real, 20))
    print(precision_recall_f1_at_k(reccomendation, real, 20))

array = np.array(array)
print("average precision is " + str(array[:,0].mean()) +", average recall is "+ str(array[:,1].mean())+ ", average f1 is " + str(array[:,2].mean()))

[0.0, 0.0, 0.0]
[0.0, 0.0, 0.0]
[0.0, 0.0, 0.0]
[0.1, 0.010309278350515464, 0.018691588785046728]
[0.0, 0.0, 0.0]
[0.0, 0.0, 0.0]
[0.0, 0.0, 0.0]
[0.0, 0.0, 0.0]
[0.0, 0.0, 0.0]
[0.0, 0.0, 0.0]
average precision is 0.01, average recall is 0.0010309278350515464, average f1 is 0.0018691588785046728


In [57]:
selected_users = [19, 182, 219, 274, 294, 480, 489, 597, 599, 600]

liste = []
for i in selected_users:
    recommended = make_recommendations_genre(id=i, uri=uri, username=username, password=password)
    liste.append(recommended)

array = []
for index, i in enumerate(selected_users):
    real = ratings_test[ratings_test['userId'] == i]['movieTitle'].tolist()
    reccomendation = liste[index]
    array.append(precision_recall_f1_at_k(reccomendation, real, 20))
    print(precision_recall_f1_at_k(reccomendation, real, 20))

array = np.array(array)
print("average precision is " + str(array[:,0].mean()) +", average recall is "+ str(array[:,1].mean())+ ", average f1 is " + str(array[:,2].mean()))

[0.0, 0.0, 0.0]
[0.05, 0.007692307692307693, 0.013333333333333334]
[0.0, 0.0, 0.0]
[0.1, 0.010309278350515464, 0.018691588785046728]
[0.0, 0.0, 0.0]
[0.0, 0.0, 0.0]
[0.05, 0.011235955056179775, 0.01834862385321101]
[0.0, 0.0, 0.0]
[0.0, 0.0, 0.0]
[0.05, 0.009708737864077669, 0.016260162601626015]
average precision is 0.025, average recall is 0.0038946278963080602, average f1 is 0.0066633708573217085


In [60]:
selected_users = [19, 182, 219, 274, 294, 480, 489, 597, 599, 600]

liste = []
for i in selected_users:
    recommended = make_recommendations_favoritedirector(id=i, uri=uri, username=username, password=password)
    liste.append(recommended)

array = []
for index, i in enumerate(selected_users):
    real = ratings_test[ratings_test['userId'] == i]['movieTitle'].tolist()
    reccomendation = liste[index]
    array.append(precision_recall_f1_at_k(reccomendation, real, 20))
    print(precision_recall_f1_at_k(reccomendation, real, 20))

array = np.array(array)
print("average precision is " + str(array[:,0].mean()) +", average recall is "+ str(array[:,1].mean())+ ", average f1 is " + str(array[:,2].mean()))

[0.05, 0.010309278350515464, 0.017094017094017096]
[0.05, 0.007692307692307693, 0.013333333333333334]
[0.1, 0.02666666666666667, 0.042105263157894736]
[0.15, 0.015463917525773196, 0.028037383177570093]
[0.05, 0.01639344262295082, 0.02469135802469136]
[0.0, 0.0, 0.0]
[0.05, 0.011235955056179775, 0.01834862385321101]
[0.0, 0.0, 0.0]
[0.0, 0.0, 0.0]
[0.0, 0.0, 0.0]
average precision is 0.045, average recall is 0.008776156791439362, average f1 is 0.014360997864071762


In [62]:
selected_users = [19, 182, 219, 274, 294, 480, 489, 597, 599, 600]

liste = []
for i in selected_users:
    recommended = make_recommendations_favoriteActor(id=i, uri=uri, username=username, password=password)
    liste.append(recommended)

array = []
for index, i in enumerate(selected_users):
    real = ratings_test[ratings_test['userId'] == i]['movieTitle'].tolist()
    reccomendation = liste[index]
    array.append(precision_recall_f1_at_k(reccomendation, real, 20))
    print(precision_recall_f1_at_k(reccomendation, real, 20))

array = np.array(array)
print("average precision is " + str(array[:,0].mean()) +", average recall is "+ str(array[:,1].mean())+ ", average f1 is " + str(array[:,2].mean()))

[0.0, 0.0, 0.0]
[0.0, 0.0, 0.0]
[0.0, 0.0, 0.0]
[0.05, 0.005154639175257732, 0.009345794392523364]
[0.0, 0.0, 0.0]
[0.0, 0.0, 0.0]
[0.05, 0.011235955056179775, 0.01834862385321101]
[0.0, 0.0, 0.0]
[0.0, 0.0, 0.0]
[0.0, 0.0, 0.0]
average precision is 0.01, average recall is 0.0016390594231437505, average f1 is 0.0027694418245734374


In [ ]:
selected_users = [19, 182, 219, 274, 294, 480, 489, 597, 599, 600]

liste = []
for i in selected_users:
    recommended = make_recommendations_mostsimilar_knn(id=i, uri=uri, username=username, password=password)
    liste.append(recommended)

array = []
for index, i in enumerate(selected_users):
    real = ratings_test[ratings_test['userId'] == i]['movieTitle'].tolist()
    reccomendation = liste[index]
    array.append(precision_recall_f1_at_k(reccomendation, real, 20))
    print(precision_recall_f1_at_k(reccomendation, real, 20))

array = np.array(array)
print("average precision is " + str(array[:,0].mean()) +", average recall is "+ str(array[:,1].mean())+ ", average f1 is " + str(array[:,2].mean()))

[0.25, 0.05154639175257732, 0.08547008547008547]
[0.4, 0.06153846153846154, 0.10666666666666667]
[0.35, 0.09333333333333334, 0.14736842105263157]
[0.4, 0.041237113402061855, 0.07476635514018691]
[0.1, 0.03278688524590164, 0.04938271604938272]
[0.65, 0.10833333333333334, 0.18571428571428572]
[0.5, 0.11235955056179775, 0.18348623853211007]
[0.3, 0.09836065573770492, 0.14814814814814814]
[0.0, 0.0, 0.0]
[0.55, 0.10679611650485436, 0.17886178861788618]
average precision is 0.35, average recall is 0.07062918414100261, average f1 is 0.11598647053913835
